# Coyote Performance Graphs

To use these Jupyter notebooks in VS Code you will need to setup a python 3.9 environment. First install the Jupyter extension in VS Code.
Make sure you have [Anaconda installed](https://docs.anaconda.com/anaconda/install/index.html).

This notebook also depends on the following environment variables so it can find your Azure cosmos performance database named `actorperfdb`.

- COSMOS_ACCOUNT_URI
- COSMOS_ACCOUNT_KEY

To setup your python environment run:

```
conda create -n coyote python=3.9
conda activate coyote
pip install -r requirements.txt
pip install -U ipykernel
code .
```

Use the VS code command palette to run `Python: Select Interpreter` and select your `coyote` python environment.

Now when you open this notebook in VS code it will connect to the Jupyter kernel so you
can execute the cells of the notebook.

When you execute the first cell you should see the output say `BokehJS 2.3.3 successfully loaded.`.

In [8]:
import math
import datetime
import pandas as pd
pd.options.display.html.table_schema = True
pd.options.display.max_rows = None
import bokeh.io
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.models.ranges import FactorRange
from bokeh.transform import factor_cmap
import bokeh.io

bokeh.io.reset_output()
bokeh.io.output_notebook()

Loading BokehJS ...

In [9]:
from azure.cosmos import CosmosClient, PartitionKey

import os
url = os.environ['COSMOS_ACCOUNT_URI']
key = os.environ['COSMOS_ACCOUNT_KEY']

cosmos_client = CosmosClient(url, credential=key)

database = cosmos_client.create_database_if_not_exists('actorperfdb')
print('Database actorperfdb created')

perfcontainer = database.create_container_if_not_exists(id='actorperfsummary', partition_key=PartitionKey(path='/PartitionKey'))
print('Container actorperfsummary opened')

commitcontainer = database.create_container_if_not_exists(id='commitlog', partition_key=PartitionKey(path='/PartitionKey'))
print('Container commitlog opened')

Database actorperfdb created
Container actorperfsummary opened
Container commitlog opened


In [10]:
perf_data = []
for item in perfcontainer.query_items(query='SELECT * FROM actorperfsummary', enable_cross_partition_query=True):
    perf_data += [item]
    
df_cosmos = pd.DataFrame(perf_data, columns=['CommitId', 'TestName', 'TimeMean', 'TimeStdDev', 'MemoryMean', 'MemoryStdDev', 'CpuMean', 'CpuStdDev'])
print(len(df_cosmos))

3134


In [11]:
commit_data = []
for item in commitcontainer.query_items(query='SELECT * FROM commitlog', enable_cross_partition_query=True):
    commit_data += [item]
    
df_commits = pd.DataFrame(commit_data, columns=['Id', 'CommitId', 'Date', 'Author'])
print(len(df_commits))

501


In [15]:
def plots_with_error_bars(width, height, title, data_frame, color='#4080A0'):
    hover = HoverTool(tooltips=[
        ('name', '@ids'),
        ('mean', '@ys ± @error')
    ])
    p = figure(title=title, tools=[hover])
    p.height = height
    p.width = width
    ys = data_frame["mean"].values
    xs = list(range(len(ys)))
    ids = list(data_frame.index)
    yerrs = data_frame["error"].values

    # create the line start and coordinates for the error bars rendered using multi_line
    err_xs = []
    err_ys = []

    for x, y, yerr in zip(xs, ys, yerrs):
        err_xs.append((x, x))
        err_ys.append((y - yerr, y + yerr))

    source = {}
    source['xs'] = xs
    source['ys'] = ys
    source['ids'] = ids  # so we can include it in the hover tooltip.
    source['error'] = yerrs

    # plot them
    p.multi_line(err_xs, err_ys, color=color, line_width=0.5)
    c = p.circle('xs', 'ys', source=source, color=color, size=5, line_alpha=0) 

    if p.y_range.start:
        p.y_range.start = min(p.y_range.start, math.floor(min(ys) / 10) * 10)
    else:
        p.y_range.start = math.floor(min(ys) / 10) * 10
    
    
    p.hover.renderers = [c]
    show(p)
        
def methods(obj):
    print('Methods:')
    print('\n'.join([x for x in dir(obj) if not x.startswith('_')]))
    

In [16]:
# Use pandas DataFrame.join method to add the real commit date to each row so 
# we can then sort by date to get the correct column order in the graphs.
# Then we groupby the TestName so we can create one plot per test.
width = 800
height = 332
print("Report generated " + datetime.date.today().strftime('%m/%d/%Y'))
df_commits.sort_values(['Date'], inplace=True)
s = df_commits.shape
most_recent = df_commits.tail(1)
print("Most recent commit: {} from {}".format(most_recent['CommitId'].values[0], most_recent['Date'].values[0]))
joined = df_cosmos.set_index('CommitId').join(df_commits.set_index('CommitId'))
joined.sort_values(['Date'], inplace=True)
grouped =joined.groupby(['TestName'])
print("{} tests found".format(len(grouped)))
datasets = []
for g in grouped:
    test_name = g[0]
    test_data = g[1].rename(columns={'TimeMean':'mean', 'TimeStdDev':'error'})    
    test_data.sort_values(['Date'], inplace=True)
    plots_with_error_bars(width, height, test_name, test_data)

Report generated 03/08/2023
Most recent commit: 5c1d96eed800cc79eb0124b381fb04c935741264 from 2023-02-27T20:41:25Z
11 tests found


C:\Users\clovett.REDMOND\AppData\Local\Temp\ipykernel_20400\3911405883.py:16: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for g in grouped:


In [ ]:
### Danger: advanced use only for cleaning up commits that have weird outlier data that you want
### to purge from the database.

outlier = "<commitid>"

for i in commit_data:
    if i['CommitId'] == outlier:
        partition = i['PartitionKey']
        print(f"deleting commit {outlier}")
        commitcontainer.delete_item(item=outlier, partition_key=partition)

for d in perf_data:
    if d['CommitId'] == outlier:
        partition = d['PartitionKey']
        id = d['id']
        print(f"deleting {id}...")
        perfcontainer.delete_item(item=id, partition_key=partition)